In [1]:
import pandas as pd

In [2]:
dt = pd.read_csv('./01.gene.protein.min20.loca.info', sep="\t", header=None)

In [3]:
dt.columns = ["Gene_id", "Genome_ID", "Contig_ID", "Start", "End", "Database", "Cls", "Cluster_ID"]
dt.head()

,Gene_id,Genome_ID,Contig_ID,Start,End,Database,Cls,Cluster_ID
0,iLABdb.g10000_00010,iLABdb.g10000,NZ_JANXLF010000001.1,211,678,NCBI,iLABdb.g10000_00010,iLABdb.c506457
1,iLABdb.g10000_00015,iLABdb.g10000,NZ_JANXLF010000001.1,693,3173,NCBI,iLABdb.g10000_00015,iLABdb.c506458
2,iLABdb.g10000_00020,iLABdb.g10000,NZ_JANXLF010000001.1,3282,4628,NCBI,iLABdb.g10001_04645,iLABdb.c558768
3,iLABdb.g10000_00025,iLABdb.g10000,NZ_JANXLF010000001.1,4903,8484,NCBI,iLABdb.g10000_00025,iLABdb.c506459
4,iLABdb.g10000_00030,iLABdb.g10000,NZ_JANXLF010000001.1,8580,12233,NCBI,iLABdb.g10000_00030,iLABdb.c506460


In [7]:
## 统计数据来源
# 统计每个 Cluster_ID 下各 Database 的计数
cluster_summary = dt.groupby(['Cluster_ID', 'Database']).size().unstack(fill_value=0)

# 分别统计仅来自 NCBI 的、仅来自 IMAU 的以及两者共有的 Cluster_ID
only_ncbi = cluster_summary[(cluster_summary['NCBI'] > 0) & (cluster_summary['IMAU'] == 0)]
only_imau = cluster_summary[(cluster_summary['IMAU'] > 0) & (cluster_summary['NCBI'] == 0)]
both_sources = cluster_summary[(cluster_summary['NCBI'] > 0) & (cluster_summary['IMAU'] > 0)]


In [ ]:
print("仅来自NCBI的Cluster_ID:")
print(len(only_ncbi.index.tolist()))
print("\n仅来自IMAU的Cluster_ID:")
print(len(only_imau.index.tolist()))
print("\n两者共有的Cluster_ID:")
print(len(both_sources.index.tolist()))

仅来自NCBI的Cluster_ID:
240722

仅来自IMAU的Cluster_ID:
147325

两者共有的Cluster_ID:
217409


In [17]:
147325/605456

0.2433289950054174

In [8]:
## 在genus level统计NCBI特有或者是IMAU特有的gene family数
genus_mapping = pd.read_csv('iLAB_genus_mapping', sep="\t")
dt_genus = pd.merge(dt, genus_mapping, left_on='Genome_ID', right_on='iLABdb_ID', how='inner')

In [9]:
dt_genus.head()

,Gene_id,Genome_ID,Contig_ID,Start,End,Database,Cls,Cluster_ID,iLABdb_ID,Genus
0,iLABdb.g10000_00010,iLABdb.g10000,NZ_JANXLF010000001.1,211,678,NCBI,iLABdb.g10000_00010,iLABdb.c506457,iLABdb.g10000,Dellaglioa
1,iLABdb.g10000_00015,iLABdb.g10000,NZ_JANXLF010000001.1,693,3173,NCBI,iLABdb.g10000_00015,iLABdb.c506458,iLABdb.g10000,Dellaglioa
2,iLABdb.g10000_00020,iLABdb.g10000,NZ_JANXLF010000001.1,3282,4628,NCBI,iLABdb.g10001_04645,iLABdb.c558768,iLABdb.g10000,Dellaglioa
3,iLABdb.g10000_00025,iLABdb.g10000,NZ_JANXLF010000001.1,4903,8484,NCBI,iLABdb.g10000_00025,iLABdb.c506459,iLABdb.g10000,Dellaglioa
4,iLABdb.g10000_00030,iLABdb.g10000,NZ_JANXLF010000001.1,8580,12233,NCBI,iLABdb.g10000_00030,iLABdb.c506460,iLABdb.g10000,Dellaglioa


In [10]:
target_databases = ['NCBI', 'IMAU']

# 统计每个 Cluster_ID 在几个 Database 出现
cluster_counts = dt_genus[dt_genus['Database'].isin(target_databases)].groupby('Cluster_ID')['Database'].nunique()

# 获取独有的 Cluster_ID
unique_clusters = cluster_counts[cluster_counts == 1].index

# 获取共有的 Cluster_ID
shared_clusters = cluster_counts[cluster_counts > 1].index

# 按 Genus 和 Database 分别计算独有 Cluster_ID 和共有 Cluster_ID
def count_unique_shared(df, database):
    unique_count = df[(df['Cluster_ID'].isin(unique_clusters)) & (df['Database'] == database)]['Cluster_ID'].nunique()
    shared_count = df[(df['Cluster_ID'].isin(shared_clusters)) & (df['Database'] == database)]['Cluster_ID'].nunique()
    total_count = df['Cluster_ID'].nunique()
    return pd.Series({'Unique_Cluster_ID_Count': unique_count, 'Shared_Cluster_ID_Count': shared_count, 'Total_Cluster_ID_Count': total_count})

genus_result = dt_genus[dt_genus['Database'].isin(target_databases)].groupby(['Genus', 'Database']).apply(lambda x: count_unique_shared(x, x['Database'].iloc[0]))

/tmp/ipykernel_275870/4157873007.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  genus_result = dt_genus[dt_genus['Database'].isin(target_databases)].groupby(['Genus', 'Database']).apply(lambda x: count_unique_shared(x, x['Database'].iloc[0]))


In [14]:
genus_result.reset_index(drop=False).head()

,Genus,Database,Unique_Cluster_ID_Count,Shared_Cluster_ID_Count,Total_Cluster_ID_Count
0,Apilactobacillus,NCBI,2181,17,2198
1,Bifidobacterium,IMAU,1578,2101,3679
2,Bifidobacterium,NCBI,5782,2101,7883
3,Carnobacterium,IMAU,3623,3212,6835
4,Carnobacterium,NCBI,7245,3352,10597


In [15]:
genus_result.reset_index(drop=False).to_excel("genus_result.xlsx")

In [9]:
## 物种特有基因
species_id = pd.read_csv('iLAB_cls.id', sep="\t")

## 合并表格
merged_data = pd.merge(dt, species_id, left_on='Genome_ID', right_on='iLABdb_ID', how='inner')


In [31]:
merged_data.to_csv("01.gene.protein.min20.loca.info.sp")

In [8]:
merged_data.head()

,Gene_id,Genome_ID,Contig_ID,Start,End,Database,Cls,Cluster_ID,iLABdb_ID,Species_cluster_ID
0,iLABdb.g10000_00010,iLABdb.g10000,NZ_JANXLF010000001.1,211,678,NCBI,iLABdb.g10000_00010,iLABdb.c506457,iLABdb.g10000,iLABdb.s053
1,iLABdb.g10000_00015,iLABdb.g10000,NZ_JANXLF010000001.1,693,3173,NCBI,iLABdb.g10000_00015,iLABdb.c506458,iLABdb.g10000,iLABdb.s053
2,iLABdb.g10000_00020,iLABdb.g10000,NZ_JANXLF010000001.1,3282,4628,NCBI,iLABdb.g10001_04645,iLABdb.c558768,iLABdb.g10000,iLABdb.s053
3,iLABdb.g10000_00025,iLABdb.g10000,NZ_JANXLF010000001.1,4903,8484,NCBI,iLABdb.g10000_00025,iLABdb.c506459,iLABdb.g10000,iLABdb.s053
4,iLABdb.g10000_00030,iLABdb.g10000,NZ_JANXLF010000001.1,8580,12233,NCBI,iLABdb.g10000_00030,iLABdb.c506460,iLABdb.g10000,iLABdb.s053


In [35]:
# 按照 Species_cluster_ID 和 Cluster_ID 统计各个分支独有的基因 family
# 统计每个 Cluster_ID 出现的分支
cluster_counts = merged_data.groupby('Cluster_ID')['Species_cluster_ID'].nunique()

# 过滤只出现一次的 Cluster_ID
unique_clusters = cluster_counts[cluster_counts == 1].index

# 从原始数据中找出这些独有的 Cluster_ID
unique_genes = merged_data[merged_data['Cluster_ID'].isin(unique_clusters)]

# 按 Species_cluster_ID 分组并统计各分支独有的 Cluster_ID
unique_gene_families_per_species = unique_genes.groupby('Species_cluster_ID')['Cluster_ID'].unique()

# 添加独有基因家族的个数统计
unique_gene_families_per_species_count = unique_gene_families_per_species.apply(len)

# 合并独有基因家族和其个数
result = pd.DataFrame({
    'Species_cluster_ID': unique_gene_families_per_species.index,
    'Unique_Gene_Families': unique_gene_families_per_species.values,
    'Unique_Family_Count': unique_gene_families_per_species_count.values
})

# 获取每个 `Species_cluster_ID` 拥有的总 `Cluster_ID` 数量
total_cluster_counts = merged_data.groupby('Species_cluster_ID')['Cluster_ID'].nunique()

# 计算独有 `Cluster_ID` 占总 `Cluster_ID` 的比例
result = result.merge(total_cluster_counts, on='Species_cluster_ID')
result.columns = ['Species_cluster_ID', 'Unique_Gene_Families', 'Unique_Family_Count', 'Total_Family_Count']
result['Unique_Family_Proportion'] = result['Unique_Family_Count'] / result['Total_Family_Count']


    Species_cluster_ID                               Unique_Gene_Families  \
0          iLABdb.s001  [iLABdb.c25956, iLABdb.c591478, iLABdb.c339887...   
1          iLABdb.s002  [iLABdb.c58023, iLABdb.c81005, iLABdb.c590299,...   
2          iLABdb.s003  [iLABdb.c387561, iLABdb.c494768, iLABdb.c96872...   
3          iLABdb.s004  [iLABdb.c315369, iLABdb.c463409, iLABdb.c33783...   
4          iLABdb.s005  [iLABdb.c530553, iLABdb.c315829, iLABdb.c42758...   
..                 ...                                                ...   
138        iLABdb.s139  [iLABdb.c150284, iLABdb.c150285, iLABdb.c15028...   
139        iLABdb.s140  [iLABdb.c394649, iLABdb.c394650, iLABdb.c39465...   
140        iLABdb.s141  [iLABdb.c366697, iLABdb.c366698, iLABdb.c36669...   
141        iLABdb.s142  [iLABdb.c352464, iLABdb.c352465, iLABdb.c35246...   
142        iLABdb.s143  [iLABdb.c426248, iLABdb.c426250, iLABdb.c42625...   

     Unique_Family_Count  Total_Family_Count  Unique_Family_Proportion  
0 

In [41]:
result.head()
result.to_csv("Species_uniq_gene_families_prop.csv")

In [38]:
## 转化为昌数据格式
# 将“宽”表转换为“长”表
result_long = result.explode('Unique_Gene_Families')
result_long = result_long.drop(columns=['Unique_Family_Count', "Total_Family_Count", "Unique_Family_Proportion"])
result_long.columns = ['Species_cluster_ID', 'Cluster_ID']

In [40]:
result_long.head()

,Species_cluster_ID,Cluster_ID
0,iLABdb.s001,iLABdb.c25956
0,iLABdb.s001,iLABdb.c591478
0,iLABdb.s001,iLABdb.c339887
0,iLABdb.s001,iLABdb.c339888
0,iLABdb.s001,iLABdb.c339889


In [5]:
## 在genus或者family level统计每个taxa特有或者独有的信息
taxa_mapping = pd.read_csv('iLABdb_taxa', sep="\t")
dt_taxa = pd.merge(dt, taxa_mapping, left_on='Genome_ID', right_on='iLABdb_ID', how='inner')

In [6]:
dt_taxa.head()

,Gene_id,Genome_ID,Contig_ID,Start,End,Database,Cls,Cluster_ID,iLABdb_ID,Species_cluster_ID,Family,Genus
0,iLABdb.g10000_00010,iLABdb.g10000,NZ_JANXLF010000001.1,211,678,NCBI,iLABdb.g10000_00010,iLABdb.c506457,iLABdb.g10000,iLABdb.s053,Lactobacillaceae,Dellaglioa
1,iLABdb.g10000_00015,iLABdb.g10000,NZ_JANXLF010000001.1,693,3173,NCBI,iLABdb.g10000_00015,iLABdb.c506458,iLABdb.g10000,iLABdb.s053,Lactobacillaceae,Dellaglioa
2,iLABdb.g10000_00020,iLABdb.g10000,NZ_JANXLF010000001.1,3282,4628,NCBI,iLABdb.g10001_04645,iLABdb.c558768,iLABdb.g10000,iLABdb.s053,Lactobacillaceae,Dellaglioa
3,iLABdb.g10000_00025,iLABdb.g10000,NZ_JANXLF010000001.1,4903,8484,NCBI,iLABdb.g10000_00025,iLABdb.c506459,iLABdb.g10000,iLABdb.s053,Lactobacillaceae,Dellaglioa
4,iLABdb.g10000_00030,iLABdb.g10000,NZ_JANXLF010000001.1,8580,12233,NCBI,iLABdb.g10000_00030,iLABdb.c506460,iLABdb.g10000,iLABdb.s053,Lactobacillaceae,Dellaglioa


In [17]:
# 统计每个 Cluster_ID 出现在多少个不同的 Family 中
family_cluster_counts = dt_taxa.groupby('Cluster_ID')['Genus'].nunique()
family_cluster_counts.head()


Cluster_ID
iLABdb.c1        1
iLABdb.c10       2
iLABdb.c100      1
iLABdb.c1000     1
iLABdb.c10000    1
Name: Genus, dtype: int64

In [18]:
# 筛选出仅在一个 Family 中出现的 Cluster_ID
unique_clusters = family_cluster_counts[family_cluster_counts == 1].index
len(unique_clusters)

576306

In [ ]:
# 确保每个 Family 下的 Cluster_ID 只被统计一次
family_unique_clusters = dt_taxa.drop_duplicates(subset=['Family', 'Cluster_ID'])

family_unique_clusters.head()

In [ ]:
# 从原始数据中筛选出这些独有的 Cluster_ID，并统计每个 Family 的数量
unique_clusters_data = family_unique_clusters[family_unique_clusters['Cluster_ID'].isin(unique_clusters)]
family_unique_counts = unique_clusters_data['Family'].value_counts()

# 输出结果
print(family_unique_counts)

In [10]:


# 计算每个 Family 有多少不同的 Cluster_ID
family_cluster_counts = family_unique_clusters['Family'].value_counts()

# 输出结果
print(family_cluster_counts)

Family
Lactobacillaceae      409249
Streptococcaceae       90413
Enterococcaceae        86349
Carnobacteriaceae      17049
Bifidobacteriaceae      9471
Name: count, dtype: int64
